# **PRUEBA TÉCNICA SOLVEX - PABLO JOSUÉ BARAHONA LUNCEY**

## Ejercicio 1:
Utiliza Pandas y el conjunto de datos público de COVID-19 proporcionado por la
Universidad de Johns Hopkins para realizar las siguientes tareas:
- Descarga los datos de COVID-19 en formato CSV o JSON desde la URL
pública.
- Carga los datos en un DataFrame de Pandas.
- Calcula el promedio de casos confirmados por día en un país específico.
- Encuentra los 10 países con la tasa de mortalidad más alta (número de
muertes / número de casos confirmados) hasta la fecha.

In [13]:
import pandas as pd

# Cargar los archivos CSV
deaths_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
confirmed_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths = pd.read_csv(deaths_csv)
confirmed = pd.read_csv(confirmed_csv)

# Transformar data
deaths = deaths.drop(columns=['Province/State', 'Lat', 'Long'])
confirmed = confirmed.drop(columns=['Province/State', 'Lat', 'Long'])

# Agrupar los datos por 'Country/Region' y sumar los valores
deaths = deaths.groupby('Country/Region').sum()
confirmed = confirmed.groupby('Country/Region').sum()

# Cálculo de promedio de casos
def avg_country(country):
    if country in confirmed.index:
        country_data = confirmed.loc[country]
        cases_avg = round(country_data.diff().mean(), 3)
        print(f"{country}: {cases_avg}")
    else:
        print(f"{country} no se encuentra en los datos de casos confirmados.")

# Top 10 mortalidad
# Suma de muertes y casos confirmados
death_total = deaths.sum(axis=1)
cases_total = confirmed.sum(axis=1)

# Calculo de tasa de mortalidad
mortality_rate = round((death_total / cases_total) * 100, 2)

# Ordenar y seleccionar los 10 países con mayor tasa de mortalidad
mortality_rate_sort = mortality_rate.sort_values(ascending=False)
top10_countries = mortality_rate_sort.head(10)
mortality_top10 = pd.DataFrame({
    'Country': top10_countries.index,
    'Mortality Rate (%)': top10_countries.values
})

In [14]:
# Promedio de casos por día por país específico
print("Promedio de casos confirmados por día en: ")
avg_country("Guatemala")
avg_country("Italy")
avg_country("US")

print("-------------------------------------------")

# Top 10 países con mayor tasa de mortalidad
print("Top 10 países con mayor tasa de mortalidad: ")
print(mortality_top10)

Promedio de casos confirmados por día en: 
Guatemala: 1084.279
Italy: 22419.886
US: 90895.535
-------------------------------------------
Top 10 países con mayor tasa de mortalidad: 
        Country  Mortality Rate (%)
0  Korea, North              600.00
1    MS Zaandam               22.20
2         Yemen               19.23
3         Sudan                7.41
4          Peru                6.83
5        Mexico                6.11
6         Syria                5.86
7         Egypt                5.16
8       Somalia                5.03
9       Ecuador                4.53


## Ejercicio 2:
Utiliza Apache Spark y un conjunto de datos público de vuelos, como el conjunto
de datos de vuelos de 2015 proporcionado por la Oficina de Estadísticas de
Transporte de EE. UU., para realizar las siguientes tareas:
- Descarga los datos de vuelos en formato CSV desde la URL pública.
- Carga los datos en un DataFrame de Spark.
- Calcula la cantidad promedio de retrasos en la llegada de vuelos en un
aeropuerto específico.
- Encuentra las 10 rutas de vuelo más populares (pares de aeropuertos) en
términos de la cantidad de vuelos.

In [15]:
! pip install pyspark

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, least, greatest

# Inicializar una sesión de Spark
spark = SparkSession.builder \
    .appName("Flight Analysis") \
    .getOrCreate()

# Cargar los datos desde el archivo CSV
file_path = "T_ONTIME_REPORTING_01.csv"
df_flights = spark.read.csv(file_path, header=True, inferSchema=True)

def delay_airport(specific_airport):
    # Filtrar los vuelos que llegan a este aeropuerto
    df_arrivals = df_flights.filter(col("DEST") == specific_airport)
    # Calcular el promedio de retrasos en la llegada
    average_arrival_delay = df_arrivals.agg(avg(col("ARR_DELAY_NEW")).alias("AverageArrivalDelay")).collect()[0]["AverageArrivalDelay"]
    print(f"Promedio de retrasos en la llegada en {specific_airport}: {average_arrival_delay:.2f} minutos")

# 10 rutas de vuelo más populares por pares de aeropuertos
def top10_airports():
    # Crear columnas para el par de aeropuertos en orden alfabético
    df_routes = df_flights.withColumn("Origen", least(col("ORIGIN"), col("DEST"))) \
                          .withColumn("Destino", greatest(col("ORIGIN"), col("DEST")))
    
    # Contar los vuelos por cada par único de aeropuertos
    df_route_counts = df_routes.groupBy("Origen", "Destino").agg(count("*").alias("Vuelos"))
    df_route_counts = df_route_counts.withColumn("Vuelos", col("Vuelos") / 2)
    
    # Ordenar las rutas por la cantidad de vuelos en orden descendente y obtener las 10 más populares
    top_10_routes = df_route_counts.orderBy(col("Vuelos").desc()).limit(10)
    
    # Mostrar las 10 rutas de vuelo más populares
    print("Top 10 rutas de vuelo más populares (pares de aeropuertos):")
    top_10_routes.show()



In [8]:
print("Restraso en aeropuertos específicos: ")
delay_airport("JFK")
delay_airport("ABI")

print("------------------------------------------")

top10_airports()

Restraso en aeropuertos específicos: 
Promedio de retrasos en la llegada en JFK: 16.35 minutos
Promedio de retrasos en la llegada en ABI: 15.41 minutos
------------------------------------------
Top 10 rutas de vuelo más populares (pares de aeropuertos):
+------+-------+------+
|Origen|Destino|Vuelos|
+------+-------+------+
|   JFK|    LAX|1144.0|
|   LAX|    SFO|1105.5|
|   LAS|    LAX| 941.5|
|   LGA|    ORD| 854.0|
|   HNL|    OGG| 820.0|
|   JFK|    SFO| 736.0|
|   ATL|    LGA| 718.0|
|   LAS|    SFO| 710.5|
|   ATL|    MCO| 707.0|
|   DFW|    ORD| 677.0|
+------+-------+------+



## Ejercicio 3:
Supongamos que tienes dos conjuntos de datos: uno en Pandas y otro en Spark. El
conjunto de datos de Pandas es una tabla llamada "datos_peliculas" con
información sobre películas:
```
ID,Título,Año
1,Película1,2020
2,Película2,2019
3,Película3,2021
4,Película4,2018
```
El conjunto de datos de Spark es un DataFrame llamado "criticas" con información
sobre las críticas de películas:
```
PeliculaID,Criticoo,Puntuacion
1,Criticoo1,4.5
2,Criticoo2,3.8
3,Criticoo1,4.2
4,Criticoo3,4.7
```
Combina estos dos conjuntos de datos para obtener una tabla que muestre el
título de la película, el año de lanzamiento y la puntuación promedio de las
críticas. Asegúrate de utilizar tanto Pandas como Spark en el proceso de
integración

In [18]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Movies and Ratings Analysis") \
    .getOrCreate()

# Crear DataFrame de Pandas
datos_peliculas = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Título': ['Película1', 'Película2', 'Película3', 'Película4'],
    'Año': [2020, 2019, 2021, 2018]
})

# Convertir el DataFrame de Pandas a un DataFrame de Spark
movies_spark_df = spark.createDataFrame(datos_peliculas)

# Crear DataFrame de Spark con las críticas
criticas = spark.createDataFrame([
    (1, 'Criticoo1', 4.5),
    (2, 'Criticoo2', 3.8),
    (3, 'Criticoo1', 4.2),
    (4, 'Criticoo3', 4.7)
], ['PeliculaID', 'Critico', 'Puntuacion'])

# Renombrar columnas en el DataFrame de películas para facilitar la combinación
movies_spark_df = movies_spark_df.withColumnRenamed('ID', 'PeliculaID')

# Unir los DataFrames de películas y críticas
combined_df = movies_spark_df.join(criticas, on='PeliculaID', how='left')

# Calcular la puntuación promedio de las críticas por película
result_df = combined_df.groupBy('Título', 'Año').agg(avg(col('Puntuacion')).alias('PromedioPts'))

# Mostrar el resultado
result_df.show(truncate=False)

+---------+----+-----------+
|Título   |Año |PromedioPts|
+---------+----+-----------+
|Película4|2018|4.7        |
|Película3|2021|4.2        |
|Película2|2019|3.8        |
|Película1|2020|4.5        |
+---------+----+-----------+



## Ejercicio Web Scraping:
Supongamos que deseas obtener el precio actual del Bitcoin (BTC) de un sitio
web de criptomonedas. La información que necesitas se encuentra en la página
de https://coinmarketcap.com/currencies/bitcoin/.
Tu tarea es escribir un script en Python que realice lo siguiente:
- Realiza una solicitud GET a la URL https://coinmarketcap.com/currencies/bitcoin/ utilizando la biblioteca `requests`.
- Analiza el contenido de la página web para extraer el precio actual del Bitcoin.
- Imprime el precio en la consola

In [19]:
import requests
from bs4 import BeautifulSoup

# URL de la página de CoinMarketCap
url = 'https://coinmarketcap.com/currencies/bitcoin/'

# Realiza una solicitud GET para obtener el contenido de la página
response = requests.get(url)

# Verifica si la solicitud fue exitosa (código de respuesta 200)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        price_element = soup.find('span', class_='sc-65e7f566-0 clvjgF base-text')
        if price_element:
            # Extrae el precio del elemento y almacénalo en una variable
            price = price_element.text
            # Imprime el precio en la consola
            print(f'El precio actual del Bitcoin es: {price}')
        else:
            print('No se pudo encontrar el elemento del precio.')
    except Exception as e:
        print(f'Ocurrió un error al extraer el precio: {e}')
else:
    print(f'Error al hacer la solicitud. Código de respuesta: {response.status_code}')


El precio actual del Bitcoin es: $58,247.03


## Teoría:

¿Cuál de las siguientes plataformas de Microsoft es una solución de análisis de big data en la nube?
- Azure Synapse Analytics

En el contexto de Azure Data Factory, ¿cuál de las siguientes actividades se utiliza para transformar y limpiar datos en un flujo de trabajo?
- Data Flow

¿Cuál de las siguientes opciones es una característica clave de Apache Spark que permite procesar datos en memoria para un rendimiento más rápido?
- Resilient Distributed Dataset (RDD)

En el contexto de Pandas, ¿cuál de las siguientes operaciones se utiliza para eliminar filas duplicadas de un DataFrame?
- df.drop_duplicates()

¿Qué lenguaje de programación se utiliza comúnmente en Azure Databricks para el procesamiento de datos y análisis?
- Scala